# Timeseries checks for quality control flags
Covers QC13-15

## Table of contents
[QC13 Daily accumulations](#QC13---Daily-accumulations)  
[QC14 Monthly accumulations](#QC14---Monthly-accumulations)  
[QC15 Streaks](#QC15---Streaks)  

See '3.3 Suspect accumulations and repeated values' in Lewis et al. (2021)

# QC13 - Daily accumulations
[Back to Index](#Table-of-contents)

#### Differences from `intenseqc`: 
- 

# QC14 - Monthly accumulations 
[Back to Index](#Table-of-contents)

#### Differences from `intenseqc`: 
- 

# QC15 - Streaks 
[Back to Index](#Table-of-contents)

#### Differences from `intenseqc`: 
- 